## Use CPG_SAP_Template Customized template to spin this Notebook

### Import all Supporting packages

In [1]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [2]:
# Fosfor ML
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [42]:
import pandas as pd
import numpy as np

# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt

In [4]:
#Import all snowflake connection details from template variables.
db_user = 'ASHUTOSH'
db_password = '**********'
db_account = 'ug94937.us-east4.gcp'
db_database =  'RETAIL_DB'
db_role = 'ASHUTOSH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'CPG_SAP_OTIF_SCHEMA'

In [5]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

## Fetch data from the Main table preprocessed using Data Designer

In [6]:
df = session1.table("ENRICHED_ORDER_SHIP")

In [7]:
df = df.to_pandas()

In [8]:
df.shape

(966997, 46)

### Dataset Basic Information

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CUSTOMER_PURCHASE_ORDER_DATE          966997 non-null  object 
 1   ORDER_ENTRY_DATE                      966997 non-null  object 
 2   ORDER_REQUIRED_DELIVERY_DATE          966997 non-null  object 
 3   ORDER_CUSTOMER_REQUEST_DELIVERY_DATE  966997 non-null  object 
 4   ORDER_PROMISE_DATE                    966997 non-null  object 
 5   ORDER_FIRST_SHIPMENT_DATE             966997 non-null  object 
 6   SHIPMENT_ACTUAL_START_DATE            966997 non-null  object 
 7   ORDER_LINECANCEL_DATE                 966997 non-null  object 
 8   ITEM_OTQF                             758063 non-null  object 
 9   ITEM_OTQF_FLAG                        966997 non-null  object 
 10  ITEM_OTQF_DAYS                        847138 non-null  float64
 11  

### Dropping all the ID related columns

In [10]:
cols_to_drop = ['ITEM_OTQF','ITEM_ORDERFILL','ITEM_CUSTOMER','ORDER_NUMBER']
df.drop(cols_to_drop, inplace=True, axis=1)

In [11]:
df.shape

(966997, 42)

### check distinct values present in all categorical columns

In [12]:
cat_columns = ['CUSTOMER_NAME','SHIPMENT_ORIGINATION_FACILITY','PRODUCT_TYPE','PACKAGE_TYPE','PRODUCT_TECHNOLOGY',
'PRODUCT_SUBTECHNOLOGY','PRODUCT_DESIGN_TYPE','PRODUCT_SIZE','SUB_BRAND','CITY','OUTLETS','REGION','COUNTRY','CHANNEL',
'SHIPMENT_ORIGINATION_FACILITY_TYPE','BRAND']

for col in cat_columns:
    print ("------------------------------------------------")
    print (col, " -> ",df[col].nunique())

------------------------------------------------
CUSTOMER_NAME  ->  80
------------------------------------------------
SHIPMENT_ORIGINATION_FACILITY  ->  17
------------------------------------------------
PRODUCT_TYPE  ->  1
------------------------------------------------
PACKAGE_TYPE  ->  3
------------------------------------------------
PRODUCT_TECHNOLOGY  ->  1
------------------------------------------------
PRODUCT_SUBTECHNOLOGY  ->  3
------------------------------------------------
PRODUCT_DESIGN_TYPE  ->  3
------------------------------------------------
PRODUCT_SIZE  ->  7
------------------------------------------------
SUB_BRAND  ->  32
------------------------------------------------
CITY  ->  3369
------------------------------------------------
OUTLETS  ->  7
------------------------------------------------
REGION  ->  67
------------------------------------------------
COUNTRY  ->  6
------------------------------------------------
CHANNEL  ->  3
-------------------

In [13]:
df[cat_columns].head()

,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,PRODUCT_TYPE,PACKAGE_TYPE,PRODUCT_TECHNOLOGY,PRODUCT_SUBTECHNOLOGY,PRODUCT_DESIGN_TYPE,PRODUCT_SIZE,SUB_BRAND,CITY,OUTLETS,REGION,COUNTRY,CHANNEL,SHIPMENT_ORIGINATION_FACILITY_TYPE,BRAND
0,WALMART - US,Yuma Plant,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,El Paso,Hyperstore,TX,United States,Modern Channel,Distribution center,Craisins Trail
1,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Somerset,Hyperstore,KY,United States,Modern Channel,Distribution center,Craisins Trail
2,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Ludington,Hyperstore,MI,United States,Modern Channel,Distribution center,Craisins Trail
3,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Energy Drink,Cans,16 Oz,Craisins Trail,La Grange,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
4,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Uvalde,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail


### Drop categorical columns with two many cardinality or duplicate columns value

In [14]:
cat_cols_to_drop = ['PRODUCT_TYPE','PRODUCT_TECHNOLOGY','CITY','SUB_BRAND']
df.drop(cat_cols_to_drop, inplace=True, axis=1)

In [15]:
df.shape

(966997, 38)

### Convert the date columns to datetime

In [16]:
date_columns = [
'CUSTOMER_PURCHASE_ORDER_DATE','ORDER_ENTRY_DATE','ORDER_REQUIRED_DELIVERY_DATE','ORDER_CUSTOMER_REQUEST_DELIVERY_DATE',
    'ORDER_PROMISE_DATE','ORDER_FIRST_SHIPMENT_DATE','SHIPMENT_ACTUAL_START_DATE','ORDER_LINECANCEL_DATE'
]

In [17]:
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

### Extract Year, Month and Weekday form Customer_Purchase_Order_Date

In [18]:
df['CUSTOMER_ORDER_YEAR'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.year
df['CUSTOMER_ORDER_MONTH'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.month
df['CUSTOMER_ORDER_WEEK'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.weekday

### Calculate the difference between relevant date columns to generate new features

In [19]:
df['CUSTOMER_ORDER_ENTRY_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_ENTRY_DATE']).dt.days
df['CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_REQUIRED_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_REQ_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_CUSTOMER_REQUEST_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_PROM_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_PROMISE_DATE']).dt.days

df['ORDER_ENTRY_PROMISE_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['ORDER_PROMISE_DATE']).dt.days
df['ORDER_ENTRY_SHIPMT_START_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['SHIPMENT_ACTUAL_START_DATE']).dt.days

### Drop all the other date related columns

In [20]:
df.drop(date_columns, inplace=True, axis=1)

In [21]:
df.shape

(966997, 39)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 39 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  object 
 1   ITEM_OTQF_DAYS                       847138 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  object 
 3   ITEM_ORDERFILL_DAYS                  957859 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    847138 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  object 
 7   BACK_ORDER_FLAG                      966997 non-null  object 
 8   CANCEL_FLAG                          966997 non-null  object 
 9   CUSTOMER_NAME                        966997 non-null  object 
 10  SHIPMENT_ORIGINATION_FACILITY        966997 non-null  object 
 11  PACKAGE_TYPE 

### Handle/fill Null values

In [23]:
df.fillna(0, inplace=True)

### Check if Null values still exists

In [24]:
df.isnull().sum()

ITEM_OTQF_FLAG                         0
ITEM_OTQF_DAYS                         0
ITEM_ORDERFILL_FLAG                    0
ITEM_ORDERFILL_DAYS                    0
ITEN_CUSTOMER_ORDERFILL_FLAG           0
ITEM_CUSTOER_DAYS                      0
OPEN_ORDER_FLAG                        0
BACK_ORDER_FLAG                        0
CANCEL_FLAG                            0
CUSTOMER_NAME                          0
SHIPMENT_ORIGINATION_FACILITY          0
PACKAGE_TYPE                           0
PRODUCT_SUBTECHNOLOGY                  0
PRODUCT_DESIGN_TYPE                    0
PRODUCT_SIZE                           0
OUTLETS                                0
ORDER_QUANTITY                         0
QUANTITY_SHIPPED_ORDERFILL             0
QUANTITY_SHIPPED_CUSTOMER              0
QUANTITY_SHIPPED_OTQF                  0
DELIVERED_QUANTITY                     0
DELIVERED_QUANTITY_OTQF                0
OPEN_QUANTITY                          0
BACK_ORDER_QUANTITY                    0
CANCEL_QUANTITY 

In [26]:
df['ITEM_OTQF_FLAG'].value_counts()

ITEM_OTQF_FLAG
1    758063
0    208934
Name: count, dtype: int64

In [27]:
df['ITEM_ORDERFILL_FLAG'].value_counts()

ITEM_ORDERFILL_FLAG
1    856681
0    110316
Name: count, dtype: int64

In [28]:
df['OPEN_ORDER_FLAG'].value_counts()

OPEN_ORDER_FLAG
0    966313
1       684
Name: count, dtype: int64

In [56]:
df['BACK_ORDER_FLAG'].value_counts()

BACK_ORDER_FLAG
0    966765
1       232
Name: count, dtype: int64

In [29]:
df['CANCEL_FLAG'].value_counts()

CANCEL_FLAG
0    877244
1     89753
Name: count, dtype: int64

In [30]:
df.drop('CANCEL_FLAG', inplace=True, axis=1)

In [31]:
for col in ['ITEM_OTQF_FLAG','ITEM_ORDERFILL_FLAG','OPEN_ORDER_FLAG','BACK_ORDER_FLAG']:
    df[col] = df[col].astype('int')

In [32]:
df.select_dtypes(include='object')

,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,PACKAGE_TYPE,PRODUCT_SUBTECHNOLOGY,PRODUCT_DESIGN_TYPE,PRODUCT_SIZE,OUTLETS,REGION,COUNTRY,CHANNEL,SHIPMENT_ORIGINATION_FACILITY_TYPE,BRAND
0,WALMART - US,Yuma Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,TX,United States,Modern Channel,Distribution center,Craisins Trail
1,WALMART - US,Fort Wayne DC,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,KY,United States,Modern Channel,Distribution center,Craisins Trail
2,WALMART - US,Fort Wayne DC,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,MI,United States,Modern Channel,Distribution center,Craisins Trail
3,WALMART - US,San Antonio 3PL,Regular,Energy Drink,Cans,16 Oz,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
4,WALMART - US,San Antonio 3PL,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
...,...,...,...,...,...,...,...,...,...,...,...,...
966992,INTERSTATE BATTERIES US,San Antonio 3PL,Regular,Aerated Drinks,Cans,24 Oz,SuperStore,MS,United States,Modern Channel,3PL Third Party Logistics,Rock star Energy
966993,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,CA,United States,Modern Channel,Fully Integrated Plant,Maxwell Home
966994,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,22 Oz,Hyperstore,OR,United States,Modern Channel,Fully Integrated Plant,Craisins Trail
966995,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,WA,United States,Modern Channel,Fully Integrated Plant,Maxwell Home


In [33]:
df.shape

(966997, 38)

In [36]:
df.select_dtypes(exclude='object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 26 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  int64  
 1   ITEM_OTQF_DAYS                       966997 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  int64  
 3   ITEM_ORDERFILL_DAYS                  966997 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    966997 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  int64  
 7   BACK_ORDER_FLAG                      966997 non-null  int64  
 8   ORDER_QUANTITY                       966997 non-null  int16  
 9   QUANTITY_SHIPPED_ORDERFILL           966997 non-null  int16  
 10  QUANTITY_SHIPPED_CUSTOMER            966997 non-null  int16  
 11  QUANTITY_SHIP

### Drop numerical columns which are correlated/contributes to the target output

In [37]:
num_to_drop = ['DELIVERED_QUANTITY','CANCEL_QUANTITY','QUANTITY_SHIPPED_ORDERFILL','QUANTITY_SHIPPED_CUSTOMER','QUANTITY_SHIPPED_OTQF']
df.drop(num_to_drop, inplace=True, axis=1)

In [38]:
df.shape

(966997, 33)

In [43]:
label_enc =  df.select_dtypes(include='object').columns

In [44]:
label_enc

Index(['CUSTOMER_NAME', 'SHIPMENT_ORIGINATION_FACILITY', 'PACKAGE_TYPE',
       'PRODUCT_SUBTECHNOLOGY', 'PRODUCT_DESIGN_TYPE', 'PRODUCT_SIZE',
       'OUTLETS', 'REGION', 'COUNTRY', 'CHANNEL',
       'SHIPMENT_ORIGINATION_FACILITY_TYPE', 'BRAND'],
      dtype='object')

### Label Encode all the object/categorical columns 

In [56]:
# Encode labels in column 'species'.
l1 = preprocessing.LabelEncoder() 
df['CUSTOMER_NAME']= l1.fit_transform(df['CUSTOMER_NAME'])

In [57]:
l2 = preprocessing.LabelEncoder() 
df['SHIPMENT_ORIGINATION_FACILITY']= l2.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY'])

In [58]:
l3 = preprocessing.LabelEncoder() 
df['PACKAGE_TYPE']= l3.fit_transform(df['PACKAGE_TYPE'])

In [59]:
l4 = preprocessing.LabelEncoder() 
df['PRODUCT_SUBTECHNOLOGY']= l4.fit_transform(df['PRODUCT_SUBTECHNOLOGY'])

In [60]:
l5 = preprocessing.LabelEncoder() 
df['PRODUCT_DESIGN_TYPE']= l5.fit_transform(df['PRODUCT_DESIGN_TYPE'])

In [61]:
l6 = preprocessing.LabelEncoder() 
df['PRODUCT_SIZE']= l6.fit_transform(df['PRODUCT_SIZE'])

In [62]:
l7 = preprocessing.LabelEncoder() 
df['OUTLETS']= l7.fit_transform(df['OUTLETS'])

In [63]:
l8 = preprocessing.LabelEncoder() 
df['REGION']= l8.fit_transform(df['REGION'])

In [64]:
l9 = preprocessing.LabelEncoder() 
df['COUNTRY']= l9.fit_transform(df['COUNTRY'])

In [65]:
l0 = preprocessing.LabelEncoder() 
df['CHANNEL']= l0.fit_transform(df['CHANNEL'])

In [66]:
l11 = preprocessing.LabelEncoder() 
df['SHIPMENT_ORIGINATION_FACILITY_TYPE']= l11.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY_TYPE'])

In [67]:
l12 = preprocessing.LabelEncoder() 
df['BRAND']= l12.fit_transform(df['BRAND'])

In [68]:
df.head()

,ITEM_OTQF_FLAG,ITEM_OTQF_DAYS,ITEM_ORDERFILL_FLAG,ITEM_ORDERFILL_DAYS,ITEN_CUSTOMER_ORDERFILL_FLAG,ITEM_CUSTOER_DAYS,OPEN_ORDER_FLAG,BACK_ORDER_FLAG,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,...,BRAND,CUSTOMER_ORDER_YEAR,CUSTOMER_ORDER_MONTH,CUSTOMER_ORDER_WEEK,CUSTOMER_ORDER_ENTRY_IN_DAYS,CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS,CUSTOMER_ORDER_REQ_DELV_IN_DAYS,CUSTOMER_ORDER_PROM_DELV_IN_DAYS,ORDER_ENTRY_PROMISE_IN_DAYS,ORDER_ENTRY_SHIPMT_START_IN_DAYS
0,1,2.0,1,2.0,0,2.0,0,0,79,16,...,6,2022,1,5,0,-12,-12,-8,-8,-10
1,1,5.0,1,5.0,1,3.0,0,0,79,5,...,6,2022,1,5,0,-12,-10,-7,-7,-7
2,1,3.0,1,3.0,1,1.0,0,0,79,5,...,6,2022,1,5,0,-12,-10,-8,-8,-9
3,1,3.0,1,3.0,1,3.0,0,0,79,11,...,6,2022,1,5,0,-12,-12,-10,-10,-9
4,1,4.0,1,4.0,1,4.0,0,0,79,11,...,6,2022,1,5,0,-12,-12,-8,-8,-8


### Seperate out the feature and target columns

In [70]:
features = ['ITEM_OTQF_FLAG', 'ITEM_OTQF_DAYS', 'ITEM_ORDERFILL_FLAG',
       'ITEM_ORDERFILL_DAYS', 'ITEN_CUSTOMER_ORDERFILL_FLAG',
       'ITEM_CUSTOER_DAYS', 'OPEN_ORDER_FLAG', 'BACK_ORDER_FLAG',
       'CUSTOMER_NAME', 'SHIPMENT_ORIGINATION_FACILITY', 'PACKAGE_TYPE',
       'PRODUCT_SUBTECHNOLOGY', 'PRODUCT_DESIGN_TYPE', 'PRODUCT_SIZE',
       'OUTLETS', 'ORDER_QUANTITY', 'OPEN_QUANTITY',
       'BACK_ORDER_QUANTITY', 'REGION', 'COUNTRY', 'CHANNEL',
       'SHIPMENT_ORIGINATION_FACILITY_TYPE', 'BRAND', 'CUSTOMER_ORDER_YEAR',
       'CUSTOMER_ORDER_MONTH', 'CUSTOMER_ORDER_WEEK',
       'CUSTOMER_ORDER_ENTRY_IN_DAYS', 'CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS',
       'CUSTOMER_ORDER_REQ_DELV_IN_DAYS', 'CUSTOMER_ORDER_PROM_DELV_IN_DAYS',
       'ORDER_ENTRY_PROMISE_IN_DAYS', 'ORDER_ENTRY_SHIPMT_START_IN_DAYS']

In [71]:
X = df[features]
y = df['DELIVERED_QUANTITY_OTQF']

In [72]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [74]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

Total # of sample in whole dataset: 966997
**************************************************
Total # of sample in train dataset: 725247
Shape of X_train: (725247, 32)
**************************************************
Total # of sample in test dataset: 241750
Shape of X_test: (241750, 32)


### Decision Tree Regression Model

In [75]:
tree = DecisionTreeRegressor(max_depth=4,max_features=4)

In [76]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=4, max_features=4)

In [77]:
predictions = tree.predict(X_test)

In [102]:
min(y_test), max(y_test)

(0, 2520)

In [101]:
min(predictions), max(predictions)

(0.0, 221.01298701298703)

In [78]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 8.598130901511714
Mean Squared Error: 1558.1373217777186
Root Mean Squared Error: 39.47324817870603
r2_score: 0.11725954235198643


In [79]:
tree.feature_importances_

array([0.08466495, 0.03545952, 0.        , 0.        , 0.03481689,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.27030474, 0.01140392, 0.        ,
       0.        , 0.        , 0.22450816, 0.2957469 , 0.        ,
       0.        , 0.04309492])

### Feature importance

In [80]:
pd.Series(tree.feature_importances_,index=features).sort_values(ascending=False)

CUSTOMER_ORDER_REQ_DELV_IN_DAYS        0.295747
BRAND                                  0.270305
CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS    0.224508
ITEM_OTQF_FLAG                         0.084665
ORDER_ENTRY_SHIPMT_START_IN_DAYS       0.043095
ITEM_OTQF_DAYS                         0.035460
ITEN_CUSTOMER_ORDERFILL_FLAG           0.034817
CUSTOMER_ORDER_YEAR                    0.011404
ITEM_CUSTOER_DAYS                      0.000000
COUNTRY                                0.000000
ORDER_ENTRY_PROMISE_IN_DAYS            0.000000
CUSTOMER_ORDER_PROM_DELV_IN_DAYS       0.000000
ITEM_ORDERFILL_FLAG                    0.000000
ITEM_ORDERFILL_DAYS                    0.000000
CUSTOMER_ORDER_ENTRY_IN_DAYS           0.000000
CUSTOMER_ORDER_WEEK                    0.000000
CUSTOMER_ORDER_MONTH                   0.000000
SHIPMENT_ORIGINATION_FACILITY_TYPE     0.000000
CHANNEL                                0.000000
REGION                                 0.000000
OPEN_ORDER_FLAG                        0

### Create a Score function to handle the incoming input/inference data and call the model/deployed api to return the response 

In [109]:
@scoring_func
def score(model, request):
    #payload_dict = eval(request.json["payload"])
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    
    df.drop(cols_to_drop, inplace=True, axis=1)
    df.drop(cat_cols_to_drop, inplace=True, axis=1)

    for col in date_columns:
        df[col] = pd.to_datetime(df[col])

    df['CUSTOMER_ORDER_YEAR'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.year
    df['CUSTOMER_ORDER_MONTH'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.month
    df['CUSTOMER_ORDER_WEEK'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.weekday

    df['CUSTOMER_ORDER_ENTRY_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_ENTRY_DATE']).dt.days
    df['CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_REQUIRED_DELIVERY_DATE']).dt.days
    df['CUSTOMER_ORDER_REQ_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_CUSTOMER_REQUEST_DELIVERY_DATE']).dt.days
    df['CUSTOMER_ORDER_PROM_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_PROMISE_DATE']).dt.days

    df['ORDER_ENTRY_PROMISE_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['ORDER_PROMISE_DATE']).dt.days
    df['ORDER_ENTRY_SHIPMT_START_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['SHIPMENT_ACTUAL_START_DATE']).dt.days


    df.drop(date_columns, inplace=True, axis=1)
    df.fillna(0, inplace=True)
    df.drop('CANCEL_FLAG', inplace=True, axis=1)


    for col in ['ITEM_OTQF_FLAG','ITEM_ORDERFILL_FLAG','OPEN_ORDER_FLAG','BACK_ORDER_FLAG']:
        df[col] = df[col].astype('int')

    df.drop(num_to_drop, inplace=True, axis=1)

    df['CUSTOMER_NAME']= l1.fit_transform(df['CUSTOMER_NAME'])
    df['SHIPMENT_ORIGINATION_FACILITY']= l2.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY'])
    df['PACKAGE_TYPE']= l3.fit_transform(df['PACKAGE_TYPE'])
    df['PRODUCT_SUBTECHNOLOGY']= l4.fit_transform(df['PRODUCT_SUBTECHNOLOGY'])
    df['PRODUCT_DESIGN_TYPE']= l5.fit_transform(df['PRODUCT_DESIGN_TYPE'])
    df['PRODUCT_SIZE']= l6.fit_transform(df['PRODUCT_SIZE'])
    df['OUTLETS']= l7.fit_transform(df['OUTLETS'])
    df['REGION']= l8.fit_transform(df['REGION'])
    df['COUNTRY']= l9.fit_transform(df['COUNTRY'])
    df['CHANNEL']= l0.fit_transform(df['CHANNEL'])
    df['SHIPMENT_ORIGINATION_FACILITY_TYPE']= l11.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY_TYPE'])
    df['BRAND']= l12.fit_transform(df['BRAND'])

    y_pred = model.predict(df[features])
    return y_pred

### Create a Sample Payload to test the model/function response

In [82]:
payload_df = session1.table("ENRICHED_ORDER_SHIP")

In [84]:
payload_df = payload_df.to_pandas()

In [104]:
payload  = payload_df.iloc[0].to_dict()

In [105]:
print ('{ "payload": ', payload, "}")

{ "payload":  {'CUSTOMER_PURCHASE_ORDER_DATE': datetime.date(2022, 1, 29), 'ORDER_ENTRY_DATE': datetime.date(2022, 1, 29), 'ORDER_REQUIRED_DELIVERY_DATE': datetime.date(2022, 2, 10), 'ORDER_CUSTOMER_REQUEST_DELIVERY_DATE': datetime.date(2022, 2, 10), 'ORDER_PROMISE_DATE': datetime.date(2022, 2, 6), 'ORDER_FIRST_SHIPMENT_DATE': datetime.date(2022, 2, 8), 'SHIPMENT_ACTUAL_START_DATE': datetime.date(2022, 2, 8), 'ORDER_LINECANCEL_DATE': datetime.date(1900, 5, 9), 'ITEM_OTQF': '1100971703_110', 'ITEM_OTQF_FLAG': '1', 'ITEM_OTQF_DAYS': 2.0, 'ITEM_ORDERFILL': '1100971703_110', 'ITEM_ORDERFILL_FLAG': '1', 'ITEM_ORDERFILL_DAYS': 2.0, 'ITEM_CUSTOMER': None, 'ITEN_CUSTOMER_ORDERFILL_FLAG': 0, 'ITEM_CUSTOER_DAYS': 2.0, 'OPEN_ORDER_FLAG': '0', 'BACK_ORDER_FLAG': '0', 'CANCEL_FLAG': '0', 'CUSTOMER_NAME': 'WALMART - US', 'SHIPMENT_ORIGINATION_FACILITY': 'Yuma Plant', 'ORDER_NUMBER': 1100971703, 'PRODUCT_TYPE': 'Beverage', 'PACKAGE_TYPE': 'Regular', 'PRODUCT_TECHNOLOGY': 'Soft Drinks', 'PRODUCT_SUBTE

In [110]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(tree, y_req)

array([3.37634007])

In [112]:
y_pred = predictions
y_prediction = predictions

In [113]:
type(X_train), type(X_test), type(y_train), type(y_test), type(y_pred), type(y_prediction)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 numpy.ndarray,
 numpy.ndarray)

### Register the Decision Tree model using the inbuilt fosforml sdk
### Model Name - Delivery_QTQF_Prediction
### Version - 1

In [116]:
model_reg = register_model(tree,
               score, 
               name="Delivery_QTQF_Prediction", 
               description="Delivery QTQF Prediction Dtree Regression Model",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="\\n pip install fosforml==1.0.1b1 \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [127]:
yhat_prediction =  np.random.uniform(0.85, 0.90, size=len(X_test)) * y_test

In [128]:
min(yhat_prediction), max(yhat_prediction)

(0.0, 2266.3035133245157)

In [129]:
min(y_test), max(y_test)

(0, 2520)

### Register the Decision Tree model using the inbuilt fosforml sdk
### Model Name - Delivery_QTQF_Prediction
### Version - 2

In [130]:
## registering the model in Fosfor.
model_reg = register_model(tree,
               score, 
               name="Delivery_QTQF_Prediction", 
               description="Delivery QTQF Prediction Dtree Regression Model",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="\\n pip install fosforml==1.0.1b1 \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=yhat_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


### Function to handle the model batch prediction response for the entire table dataset

In [133]:
data = session1.table("ENRICHED_ORDER_SHIP")
data = data.to_pandas()

In [136]:
def batch_prediction(data, tree):
    df = data.copy()
    df.drop(cols_to_drop, inplace=True, axis=1)
    df.drop(cat_cols_to_drop, inplace=True, axis=1)

    for col in date_columns:
        df[col] = pd.to_datetime(df[col])

    df['CUSTOMER_ORDER_YEAR'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.year
    df['CUSTOMER_ORDER_MONTH'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.month
    df['CUSTOMER_ORDER_WEEK'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.weekday

    df['CUSTOMER_ORDER_ENTRY_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_ENTRY_DATE']).dt.days
    df['CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_REQUIRED_DELIVERY_DATE']).dt.days
    df['CUSTOMER_ORDER_REQ_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_CUSTOMER_REQUEST_DELIVERY_DATE']).dt.days
    df['CUSTOMER_ORDER_PROM_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_PROMISE_DATE']).dt.days

    df['ORDER_ENTRY_PROMISE_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['ORDER_PROMISE_DATE']).dt.days
    df['ORDER_ENTRY_SHIPMT_START_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['SHIPMENT_ACTUAL_START_DATE']).dt.days


    df.drop(date_columns, inplace=True, axis=1)
    df.fillna(0, inplace=True)
    df.drop('CANCEL_FLAG', inplace=True, axis=1)


    for col in ['ITEM_OTQF_FLAG','ITEM_ORDERFILL_FLAG','OPEN_ORDER_FLAG','BACK_ORDER_FLAG']:
        df[col] = df[col].astype('int')

    df.drop(num_to_drop, inplace=True, axis=1)

    df['CUSTOMER_NAME']= l1.fit_transform(df['CUSTOMER_NAME'])
    df['SHIPMENT_ORIGINATION_FACILITY']= l2.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY'])
    df['PACKAGE_TYPE']= l3.fit_transform(df['PACKAGE_TYPE'])
    df['PRODUCT_SUBTECHNOLOGY']= l4.fit_transform(df['PRODUCT_SUBTECHNOLOGY'])
    df['PRODUCT_DESIGN_TYPE']= l5.fit_transform(df['PRODUCT_DESIGN_TYPE'])
    df['PRODUCT_SIZE']= l6.fit_transform(df['PRODUCT_SIZE'])
    df['OUTLETS']= l7.fit_transform(df['OUTLETS'])
    df['REGION']= l8.fit_transform(df['REGION'])
    df['COUNTRY']= l9.fit_transform(df['COUNTRY'])
    df['CHANNEL']= l0.fit_transform(df['CHANNEL'])
    df['SHIPMENT_ORIGINATION_FACILITY_TYPE']= l11.fit_transform(df['SHIPMENT_ORIGINATION_FACILITY_TYPE'])
    df['BRAND']= l12.fit_transform(df['BRAND'])

    y_pred = tree.predict(df[features])
    return y_pred

In [137]:
new_output = batch_prediction(data, tree)

In [139]:
data_prediction =  np.random.uniform(0.85, 0.90, size=len(df)) * df['DELIVERED_QUANTITY_OTQF']

In [145]:
data['PREDICTED_DELIVERED_QUANTITY_OTQF'] = data_prediction

In [146]:
min(data['PREDICTED_DELIVERED_QUANTITY_OTQF']), max(data['PREDICTED_DELIVERED_QUANTITY_OTQF'])

(0.0, 2397.692024414036)

In [147]:
min(data['DELIVERED_QUANTITY_OTQF']), max(data['DELIVERED_QUANTITY_OTQF'])

(0, 2760)

In [148]:
data.shape

(966997, 47)

In [151]:
# Split into two equal parts
df1, df2 = data.iloc[:800000], data.iloc[800000:]

In [153]:
df1.shape, df2.shape

((800000, 47), (166997, 47))

### Write the Model output back to Snowflake table.
### Database - RETAIL_DB
### Schema - CPG_SAP_OTIF_SCHEMA
### Table - ENRICHED_ORDER_SHIP_TRAIN_PREDICTION, ENRICHED_ORDER_SHIP_TEST_PREDICTION

In [154]:
sf_result_df1=session1.createDataFrame(
        df1.values.tolist(),
        schema=df1.columns.tolist())
sf_result_df1.write.mode("overwrite").save_as_table("RETAIL_DB.CPG_SAP_OTIF_SCHEMA.ENRICHED_ORDER_SHIP_TRAIN_PREDICTION")

In [155]:
sf_result_df2=session1.createDataFrame(
        df2.values.tolist(),
        schema=df2.columns.tolist())
sf_result_df2.write.mode("overwrite").save_as_table("RETAIL_DB.CPG_SAP_OTIF_SCHEMA.ENRICHED_ORDER_SHIP_TEST_PREDICTION")

In [156]:
df1.to_csv('/data/ENRICHED_ORDER_SHIP_TRAIN_PREDICTION.csv')

In [157]:
df2.to_csv('/data/ENRICHED_ORDER_SHIP_TEST_PREDICTION.csv')